In [7]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [8]:
bw.projects.set_current('GSA for setac')
co = bw.Database('CH consumption 1.0')
food = co.search('food')
assert len(food)==1
food = food[0]

In [9]:
list_functional_units = []
for exc in food.exchanges():
    if exc['type'] != 'production':
        list_functional_units.append({exc['input']: exc['amount']})
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]

In [10]:
bw.calculation_setups['food_contribution_analysis'] = {'inv':list_functional_units, 'ia':list_methods}

In [11]:
%%time
myMultiLCA = bw.MultiLCA('food_contribution_analysis')

CPU times: user 6.99 s, sys: 169 ms, total: 7.16 s
Wall time: 1.55 s


In [12]:
lcia_unit = bw.Method(list_methods[0]).metadata['unit']

In [13]:
fus = [bw.get_activity(list(el.keys())[0])['name'] for el in list_functional_units]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['exchanges'] = [list(fu.values())[0] for fu in list_functional_units]
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_functional_units]

In [14]:
df.sort_values(lcia_unit, ascending=False, inplace=True)

In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

                                                     kg CO2-Eq  exchanges  \
Hard and semi-hard cheese                           307.664373  32.790849   
Sausages, cold meat and pies                        264.869498  32.736453   
Beef                                                261.201262  23.888668   
Chocolate                                           243.246316  20.076453   
Pork, fresh or frozen                               175.311427  19.597377   
Ham, bacon and other cured or smoked pork           174.649962  20.571595   
Fresh, soft and melted cheese                       151.621308  16.159789   
Pure and ground coffee                              146.230020  18.044207   
Veal                                                108.447046   9.918235   
Other boiled, dried, cured or smoked meat            85.065609   6.807338   
Poultry, fresh or frozen                             52.604973  21.784385   
Other eatable meat products incl. offal, fresh ...   50.621454   9.526424   

In [16]:
df_sorted = df.sort_values(lcia_unit, ascending=False)
# df_sorted['kg CO2-Eq'].plot.barh(figsize=(20,40))

# Plot contributions

In [17]:
import plotly.graph_objects as go

In [18]:
product_names_dict = {
    "Hard and semi-hard cheese": 
        "Hard cheese",
    "Sausages, cold meat and pies":
        "Sausages",
    "Fresh, soft and melted cheese":
        "Soft and melted cheese",
    'Other boiled, dried, cured or smoked meat':
        'Dried or smoked meat',
    'Other sugary or cocoa-based foods': 
        'Other sugary or cocoa-based foods', 
    'Oranges and other citrus fruits': 
        'Oranges and citrus fruits',
    'Other flours and meals, starches, semolina, flakes and grains':
        'Other flours, flakes and grains, etc',
    'Milk-based beverages and other similar milk-based products':
        'Milk-based beverages and products',
    'Bakery products':
        'Bakery products (sweet and salty)', 
    'Potatoes-based products and other tuber vegetables':
        'Potatoes-based products',
    'Nuts, other edible nuts and seeds oleaginous fruits':
        'Nuts',
    'Tinned or processed vegetables and mushrooms':
        'Tinned or processed vegetables', 
    'Other vegetable oils and edible animal fats':
        'Vegetable oils and animal fats',
    'Tinned meat and other meat-based preparations':
        'Meat-based preparations', 
    'Beets and other root vegetables':
        'Beets and root vegetables',
    'Instant coffee and coffee surrogates':
        'Instant coffee and surrogates', 
    'Green salads and other leafy vegetables':
        'Green salads',
    'Other eatable meat products incl. offal, fresh and frozen':
        'Other meat products',
    'Ham, bacon and other cured or smoked pork':
        'Ham and bacon',
    }


In [23]:
product_names_red = ["Soft and melted cheese",]
product_names_green = ["Soft and melted cheese",]
product_names_blue

In [24]:
df_plotting = df_sorted[:26]
product_names_plot = [product_names_dict.get(name, name) for name in df_plotting.index]
n_acts = len(df_plotting)

In [25]:
colors = ["#636EFA"]*n_acts
for i,prod in enumerate(product_names_plot):
    if prod in product_names_red:
        colors[i] = "#EF553B"
    elif prod in product_names_green:
        colors[i] = "#00CC96"
    elif prod in product_names_blue:
        colors[i] = "#636EFA"
    elif prod in product_names_purple:
        colors[i] = "#AB63FA"
    elif prod in product_names_orange:
        colors[i] = "#FFA15A"

In [26]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        y=product_names_plot,
        x=df_plotting[lcia_unit].values,
        orientation="h",
        marker_color=colors,
    ),  
)
fig.update_layout(
    width=500,
    height=n_acts*22,
    yaxis=dict(autorange="reversed"),
    xaxis_title="LCIA scores, [kg CO2-eq]",
)
fig.show()
fig.write_image("food_contribution.pdf")

# BW2 contribution analysis

In [ ]:
import bw2analyzer

In [ ]:
lca = bw.LCA({food: 1}, list_methods[0])

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
ca = bw2analyzer.ContributionAnalysis()

In [ ]:
ca.annotated_top_processes(lca)

# Contribution analysis for sectors

In [ ]:
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]
list_fus = []
for cat_of_interest in category_activities:
    list_fus.append({co.search(cat_of_interest)[0]: 1})
    
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}

%%time
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')

lcia_unit = bw.Method(list_methods[0]).metadata['unit']

fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]

df.sort_values(lcia_unit, ascending=False, inplace=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)